5. Aplicar el Algoritmo REV2 al siguiente set de datos de reviews de productos de Amazon, para detectar potenciales usuarios maliciosos y otros ciertamente honestos. Por simplificación (y unificación de criterios), considerar $γ_1 = γ_2 = 0.5$. Obtener aquellos usuarios cuya justicia (fairness) es menor o igual a 0.2 (son maliciosos) y tienen al menos 5 reviews, así como la proporción de nodos que son extremadamente justos: aquellos con justicia mayor o igual a 0.9, y con al
menos 10 reviews (aristas de salida). 

**[2 Puntos]**

In [2]:
import pandas as pd
import numpy as np

In [3]:
reviews_df = pd.read_csv('../data/ratings_Electronics.csv')

In [4]:
reviews_df.head()

,User ID,Product ID,Rating,Time Stamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [5]:
reviews_df["Rating"] = np.where(reviews_df["Rating"] < 2.5, -1, 1)

In [6]:
reviews_df.head()

,User ID,Product ID,Rating,Time Stamp
0,AKM1MP6P0OYPR,0132793040,1,1365811200
1,A2CX7LUOHB2NDG,0321732944,1,1341100800
2,A2NWSAGRHCP8N5,0439886341,-1,1367193600
3,A2WNBOD3WNDNKT,0439886341,1,1374451200
4,A1GI0U4ZRJA8WN,0439886341,-1,1334707200


In [7]:
import networkx as nx

In [8]:
reviews_nx = nx.from_pandas_edgelist(reviews_df, source='User ID', target='Product ID', edge_attr='Rating',create_using=nx.DiGraph())

In [9]:
fairness = {}
reliability = {}
goodness = {}

In [10]:
y1 = 0.5
y2 = 0.5

In [11]:
def reliability_score(y1,y2,fairness,score,goodness):
    return (1/(y1+y2)) * (y1*fairness + y2*(1-((abs(score-goodness))/2)))

In [12]:
for i in range(10):
    # actualizar el fairness 
    for user in reviews_nx:
        if reviews_nx.out_degree(user) > 1: # si no tiene salidas es un producto
            reliability_sum = 0
            for product in reviews_nx[user].keys():
                reliability_sum += reliability.get((user,product),1)
            fairness[user] = reliability_sum / reviews_nx.out_degree(user)

    # actualizar el goodness
    for product in reviews_nx:
        if reviews_nx.out_degree(product) == 0: # si no tiene salidas es un producto
            reliability_x_score_sum = 0
            for user,_ in reviews_nx.in_edges(product):
                reliability_x_score_sum += reliability.get((user,product),1)*reviews_nx[user][product]["Rating"]
            goodness[product] = reliability_x_score_sum/reviews_nx.in_degree(product)

    # actualizar el reliability
    for user in reviews_nx:
        if reviews_nx.out_degree(user) > 1: # si no tiene salidas es un producto
            for product in reviews_nx[user].keys():
                reliability[(user,product)] = reliability_score(y1,y2,fairness[user],reviews_nx[user][product]["Rating"],goodness[product])
    print("Iteración: ", i)

Iteración:  0
Iteración:  1
Iteración:  2
Iteración:  3
Iteración:  4
Iteración:  5
Iteración:  6
Iteración:  7
Iteración:  8
Iteración:  9


In [13]:
usuarios_maliciosos = {}
usuarios_justos = {}
for user in reviews_nx:
    if reviews_nx.out_degree(user) > 1: # si no tiene salidas es un producto
        if fairness[user] < 0.2 and reviews_nx.out_degree(user) > 5:
            usuarios_maliciosos[user] = fairness[user]
        elif fairness[user] > 0.9 and reviews_nx.out_degree(user) > 10:
            usuarios_justos[user] = fairness[user]          

In [26]:
usuarios_maliciosos

{'A2L3ZOXJL8OR9G': 0.19908269520109817,
 'A3B3L6ENHSO3DN': 0.1865228386054922,
 'A2PEWTOJF0LT8J': 0.17943795675103957,
 'A3TWCBY131EVQ0': 0.16305536351672925,
 'A16PNERQT4XSN3': 0.17874461137213335,
 'A2VWNSDCFK3OOV': 0.17887248264136432,
 'A2F3NG6RUGZ5K2': 0.16186789894431539,
 'A24HK9CEPMHJRV': 0.14959376955713,
 'ARTEBG4476RHV': 0.1715880340531254,
 'A3OJDREFH6BAFD': 0.14899753762351262,
 'AF6JIG0PLFBLC': 0.12166692800397565,
 'A1GNHV6L0QRZ9O': 0.1818600279699439,
 'A28ZX0WY495DAI': 0.15846049397428713,
 'A3ECP9FPY96ST2': 0.15933997267606545,
 'A1WYFF9WII2SBE': 0.19604570339573854,
 'A1CJTPH6BQERSN': 0.1938017559665863,
 'A24BBVS0WQ8OD2': 0.1805796421478519,
 'A122BP6MJZL20L': 0.13983055461279612,
 'A9TWDRS2IVXRH': 0.1888637525611068,
 'A3VNX3A8V805AL': 0.16150331356117112,
 'A2OJM0F1A56TM6': 0.17024588196697868,
 'A1JLNEVY9J19MU': 0.17549606044684504}

In [24]:
len(usuarios_maliciosos)/len(reviews_nx.nodes())*100

0.0004703168096786069

In [27]:
usuarios_justos

{'A1THJB1ORM3U2I': 0.9021875949808175,
 'A35FTKYO9UQZWF': 0.901241657768008,
 'AQTRY3K8P7GCO': 0.9132355237573527,
 'ASOQROO9OD3XG': 0.9181227028298952,
 'A3V6YL38FIEDP': 0.9081760116121995,
 'AKT8TGIT6VVZ5': 0.9193184991805302,
 'A27BPKU7ZJ3NOO': 0.9016651334553171,
 'A2V0SRJJJ3PD91': 0.9013408059385766,
 'AYOIM6S23UDAH': 0.9040245477564934,
 'AOOZR8IN9NYTA': 0.9118139938797415,
 'A29S8I5VT9TKZA': 0.9073594426171243,
 'A1BPJ55PLYUA77': 0.9032846204545326,
 'A148IYQMD3J6YD': 0.9135937249837525,
 'A12RHQPWEZWAAH': 0.9086246596110915,
 'A1SIX968W2WNZM': 0.9027616153243123,
 'A1R6RKKE44AYA8': 0.9088216851605071,
 'A2FC3ME27RF1N': 0.9137163616256901,
 'A21QQB0GM1B9BS': 0.9056754908150084,
 'A1PYTLRARR1W0K': 0.9444559568708671,
 'AV1B3S6ONDIWL': 0.9072056063482964,
 'A1E9RGETV1X5LK': 0.9202916402335234,
 'A2US3LOX0TWTHB': 0.9112326634003161,
 'A1PZEA97302PQZ': 0.901275732909907,
 'A31EE4G7VHLJIF': 0.9142484219382923,
 'A2YA18JC0YGZL8': 0.9083370839720554,
 'AUJWP901VCRQ9': 0.920970195746985

In [28]:
len(usuarios_justos)

600

In [25]:
len(usuarios_justos)/len(reviews_nx.nodes())*100

0.012826822082143825

Utilizando el algoritmo de REV2 se obtuvieron dos diccionarios con los nombres de usuario y el valor de fariness.

Considerando el fairness menor a 0.2 como usuario malicioso, se encontraron 22 usuarios, no mas del 0.0005% de la red

Considerando el fairness mayor a 0.9 como usuario justo, se encontraron 600 usuarios, 0.02% de la red.